# Practice for Image Processing Logic
Using references from previous data wrangling labs and inclass1 to develop our functions for image processing.

In [3]:
#imports
#importing os to deal with file system
import os

#imports for data wrangling
import numpy as np
import pandas as pd

#imports to allow us to work with our images in matplot lib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

#importing open CV
try:
    import cv2
except:
    !pip install opencv-python
    import cv2

#import Pillow
try:
    from PIL import Image, ImageFilter
except:
    !pip install opencv-python
    from PIL import Image, ImageFilter


# Thresholding
Following the guide from InClass1, 


# Edge Detection


Documentation: 
OpenCV - Canny Edge Detection Documentation:
- https://docs.opencv.org/4.x/da/d22/tutorial_py_canny.html


# Contour Extraction

Helpful Resources:
- https://pyimagesearch.com/2021/05/12/adaptive-thresholding-with-opencv-cv2-adaptivethreshold/
- https://pillow.readthedocs.io/en/stable/handbook/concepts.html#coordinate-system
- 